## Easy

Use any BBC news article. Analyze it with Name entity recognizer of Spacy, generate Markdown or HTML file, with link of named entity to specified Wikipedia page.

In [1]:
import spacy, requests, re

In [2]:
filename = "./sample_data/text.txt"
text = "  ".join(open(filename).readlines())

In [3]:
def getWiki(text):
  text = re.sub(r'[^\w]', ' ', text)
  return requests.get(f'https://en.wikipedia.org/w/api.php?action=opensearch&search={text}&limit=1&namespace=0&format=json').json()[3]

In [4]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
res = text
for i in sorted(set([i.text for i in doc.ents]), key=lambda x: len(x), reverse=True):
    link = getWiki(i)
    if len(link):
      rtext = f'[{i}]({link[0]})'
      res = re.sub(f'(?<![\[\(\/]){i}(?![\]\)])', rtext, res)
print(res)

[ely](https://en.wikipedia.org/wiki/Ely), akin to “spoiled”, or “Mama’s boy”.
  
  But with burgers’ illustrious rise to fame, a homegrown bun-kebab’s identity emerged almost in antithesis. The bun-kebab is what the burger is not: rooted in, not removed from, reality; and owned rather than mocked. As [one](https://en.wikipedia.org/wiki/One) [Pakistani](https://en.wikipedia.org/wiki/Pakistani) wrote in an article in [The Express Tribune](https://en.wikipedia.org/wiki/The_Express_Tribune), “Dear burgers, I am a bun kebab, and proud of it.”
  
  Dear burgers, I am a bun kebab, and proud of it
  
  However, with time, it seems that the distinction, especially at a linguistic level, is becoming increasingly blurred. As another [Pakistani](https://en.wikipedia.org/wiki/Pakistani) put it, placing a nation’s identity [between two](https://en.wikipedia.org/wiki/Between_Two_Ferns_with_Zach_Galifianakis) slices of bread is a baffling predicament, especially when the terms start being used interch

In [5]:
f = open("res.md", "a")
f.write(res)
f.close()

## Medium

6th Hard task, use Gensim's FastText word vector and compare it with result of 6th Hard Task

In [6]:
import gensim.downloader as api
import numpy as np
import pandas as pd

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [8]:
w2v = api.load("fasttext-wiki-news-subwords-300")

In [9]:
df = pd.read_csv('./sample_data/IMDB Dataset.csv',error_bad_lines=False, engine="python")

In [10]:
df['tokens'] = df['review'].apply(lambda doc: list(filter(None,re.split(r'[^\w]', doc))))

In [11]:
df['w2v'] = df['tokens'].apply(lambda tokens: [w2v[i.lower()] for i in tokens if i.lower() in w2v])
df['w2v'].head()

0    [[0.019569, 0.0051348, 0.024567, -0.0037714, -...
1    [[-0.0079206, -0.095293, 0.031266, 0.017734, -...
2    [[-0.060418, 0.069955, 0.053173, 0.052922, 0.0...
3    [[0.037373, 0.030506, -0.0062058, 0.011153, 0....
4    [[-0.0105, -0.0053143, 0.0080346, 0.00036603, ...
Name: w2v, dtype: object

In [12]:
df['mean'] = df['w2v'].apply(lambda x: np.mean(x, axis=0))
df['mean'].head()

0    [0.0028416538, 0.002920228, 0.022609876, 0.010...
1    [0.0062881964, -0.0074260477, 0.031216867, 0.0...
2    [-0.00534748, -0.0035081736, 0.024448004, 0.01...
3    [0.008908586, -0.0065263174, 0.02462708, 0.018...
4    [0.015048433, -0.0055712895, 0.029336138, 0.01...
Name: mean, dtype: object

In [13]:
def posneg(x):
    if x=="negative":
        return 0
    elif x=="positive":
        return 1
    return x

filtered_score = df["sentiment"].map(posneg)
df["score"] = filtered_score

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df["mean"], df["score"], test_size=0.2)

In [15]:
logReg = LogisticRegression(fit_intercept=True, n_jobs=-1)
logReg.fit(list(X_train), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
y_pred_test = logReg.predict(list(X_test))

In [17]:
print(f"Accuracy: {accuracy_score(y_test, y_pred_test)}")
print(f"Precision: {precision_score(y_test, y_pred_test)}")
print(f"Recall: {recall_score(y_test, y_pred_test)}")
print(f"F1: {f1_score(y_test, y_pred_test)}")

Accuracy: 0.8068
Precision: 0.7995217218015146
Recall: 0.8124746861077359
F1: 0.805946163117718


Compared to the previous results:  
Accuracy: 0.7169421487603306  
Precision: 0.7489361702127659  
Recall: 0.6929133858267716  
F1: 0.719836400817996  

## Hard

Solve the following problem, and use Logistic regression or neural network to predict Named entities

[https://www.kaggle.com/abhinavwalia95/entity-annotated-corpushttps://www.kaggle.com/abhinavwalia95/entity-annotated-corpus](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)

In [18]:
import zipfile
from sklearn.preprocessing import OneHotEncoder

In [19]:
filename = './sample_data/archive.zip'
zip = zipfile.ZipFile(filename, 'r')
files = zip.namelist()

In [32]:
df = pd.read_csv(zip.open(files[1]),encoding='latin1',error_bad_lines=False, engine="python")

df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [33]:
df['w2v'] = df['Word'].apply(lambda x: w2v[x.lower()] if x.lower() in w2v else np.zeros(w2v.vector_size))
df = df.iloc[:400000, :]
df['w2v'].head()

0    [-0.0051715, -0.011219, 0.062319, 0.016572, -0...
1    [0.0068832, -0.063845, 0.080359, 0.026035, -0....
2    [0.022512, 0.018823, 0.0058266, 0.0051777, -0....
3    [-0.016487, -0.058297, 0.0095203, -0.0012367, ...
4    [-0.012286, 0.059406, 0.011795, 0.075737, 0.06...
Name: w2v, dtype: object

In [34]:
df['w2v'].values.shape

(400000,)

In [35]:
enc = OneHotEncoder(handle_unknown='ignore')
pos = enc.fit_transform(df[["POS"]]).toarray()
print(pos.shape)
print(np.vstack(df['w2v'].values).shape)
X = np.concatenate((np.vstack(df['w2v'].values), pos), axis=1)

(400000, 41)
(400000, 300)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, df["Tag"], test_size=0.2)

In [37]:
logReg = LogisticRegression(fit_intercept=True, n_jobs=-1)
logReg.fit(list(X_train), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
y_pred_test = logReg.predict(list(X_test))

In [40]:
print(f"Accuracy: {accuracy_score(y_test, y_pred_test)}")
# print(f"Precision: {precision_score(y_test, y_pred_test)}")
# print(f"Recall: {recall_score(y_test, y_pred_test)}")
# print(f"F1: {f1_score(y_test, y_pred_test)}")

Accuracy: 0.94315
